# Calculation of reserve requirements

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from pathlib import Path

# Calc reserve requirements for BaseCase

In [ ]:
import pandas as pd
import os
from pathlib import Path


def calculate_reserve_requirements(intermittent_folder, demand_file, RI_file, NTC_file, output_folder,v, x, y, z, d):
    areas = ["NO1", "NO2", "NO3", "NO4", "NO5", "SE1", "SE2", "SE3", "SE4"]

    df_reserve_RI = pd.read_csv(RI_file, index_col="Area")
    df_demand = pd.read_csv(demand_file, index_col="Timestamp")
    df_ntc = pd.read_csv(NTC_file)

  
    df_up_reserve = pd.DataFrame(index=df_demand.index)
    df_down_reserve = pd.DataFrame(index=df_demand.index)

    for area in areas:
        intermittent_file = os.path.join(intermittent_folder, f"{area}_intermittent_energy.csv")
        df_intermittent = pd.read_csv(intermittent_file, index_col="Timestamp")

        intermittent_reserve = (
            df_intermittent["Onshore"] * v +
            df_intermittent["Offshore"] * x +
            df_intermittent["PV"] * y +
            df_intermittent["Run_of_river"] * z
        )

        demand = df_demand[area] * d
        up_reserve = df_reserve_RI.loc[area, "Up"]
        down_reserve = df_reserve_RI.loc[area, "Down"]

        import_to_area = df_ntc.loc[df_ntc["Area"].isin(areas), area].sum()
        if area == "SE4":
            import_to_area += 1700

        if area == "SE3":
            import_to_area += 1200
        if area == "SE1":
            import_to_area += 1100
        import_reduction = import_to_area * 0.10
       

        df_up_reserve[area] = intermittent_reserve + demand + up_reserve - import_reduction
        df_down_reserve[area] = intermittent_reserve + demand + down_reserve - import_reduction

    df_up_reserve[df_up_reserve <= 0] = 0
    df_down_reserve[df_down_reserve <= 0] = 0
    
    df_up_reserve = df_up_reserve.round(4)
    df_down_reserve = df_down_reserve.round(4)

    up_output_file = os.path.join(output_folder, "Up_reserve_BaseCase.csv")
    down_output_file = os.path.join(output_folder, "Down_reserve_BaseCase.csv")

    df_up_reserve.to_csv(up_output_file)
    df_down_reserve.to_csv(down_output_file)

    print(f"Saved requirement for up regulation: {up_output_file}")
    print(f"Saved requirement for down regulation: {down_output_file}")
    print("Done!")


calculate_reserve_requirements(
    intermittent_folder=Path("2024_Intermittent"),
    demand_file=Path("2024_Data/Consumption_2024.csv"),
    RI_file=Path("Reserve_req_2024/RI_req.csv"),
    NTC_file=Path("2024_Data/NTC.csv"),
    output_folder=Path("Reserve_req_2024"),
    v=0.1449,           # Share of Onshore 
    x=0.1993,           # Share of offshore
    y=0.1696,           # Share of PV
    z=0.01,             # Share of Run_of_river
    d=0.00825,          # Share of demand 
)

# Calc reserve requirements for UpperBound

In [ ]:
import pandas as pd
import os
from pathlib import Path


def calculate_reserve_requirements(intermittent_folder, demand_file, RI_file, NTC_file, output_folder,v, x, y, z, d):
    areas = ["NO1", "NO2", "NO3", "NO4", "NO5", "SE1", "SE2", "SE3", "SE4"]

    df_reserve_RI = pd.read_csv(RI_file, index_col="Area")
    df_demand = pd.read_csv(demand_file, index_col="Timestamp")

  
    df_up_reserve = pd.DataFrame(index=df_demand.index)
    df_down_reserve = pd.DataFrame(index=df_demand.index)

    for area in areas:
        intermittent_file = os.path.join(intermittent_folder, f"{area}_intermittent_energy.csv")
        df_intermittent = pd.read_csv(intermittent_file, index_col="Timestamp")

        intermittent_reserve = (
            df_intermittent["Onshore"] * v +
            df_intermittent["Offshore"] * x +
            df_intermittent["PV"] * y +
            df_intermittent["Run_of_river"] * z
        )
        demand = df_demand[area] * d
        up_reserve = df_reserve_RI.loc[area, "Up"]
        down_reserve = df_reserve_RI.loc[area, "Down"]

        df_up_reserve[area] = intermittent_reserve + demand + up_reserve 
        df_down_reserve[area] = intermittent_reserve + demand + down_reserve

    df_up_reserve[df_up_reserve <= 0] = 0
    df_down_reserve[df_down_reserve <= 0] = 0
    
    df_up_reserve = df_up_reserve.round(4)
    df_down_reserve = df_down_reserve.round(4)

    up_output_file = os.path.join(output_folder, "Up_reserve_UpperBound.csv")
    down_output_file = os.path.join(output_folder, "Down_reserve_UpperBound.csv")

    df_up_reserve.to_csv(up_output_file)
    df_down_reserve.to_csv(down_output_file)

    print(f"Saved requirement for up regulation: {up_output_file}")
    print(f"Saved requirement for down regulation: {down_output_file}")
    print("Done!")


calculate_reserve_requirements(
    intermittent_folder=Path("2024_Intermittent"),
    demand_file=Path("2024_Data/Consumption_2024.csv"),
    RI_file=Path("Reserve_req_2024/RI_req.csv"),
    NTC_file=Path("2024_Data/NTC.csv"),
    output_folder=Path("Reserve_req_2024"),
    v=0.1449,           # Share of Onshore 
    x=0.1993,           # Share of offshore
    y=0.1696,           # Share of PV
    z=0.01,             # Share of Run_of_river
    d=0.00825,          # Share of demand 
)

# Calc residual load - common to all scenarios

In [ ]:
import pandas as pd
import os
from pathlib import Path


def residual_load(intermittent_folder, demand_file, output_folder):
   
    areas = ["NO1", "NO2", "NO3", "NO4", "NO5", "SE1", "SE2", "SE3", "SE4"]
    
    df_residual = pd.DataFrame()
    df_demand = pd.read_csv(demand_file, index_col = "Timestamp")
    
    for area in areas:
        
        intermittent_file = os.path.join(intermittent_folder, f"{area}_intermittent_energy.csv")
        df_intermittent = pd.read_csv(intermittent_file, index_col="Timestamp")

        
        df_reserve_intermittent = pd.DataFrame(index=df_intermittent.index)
        df_reserve_intermittent[area] = (
            df_intermittent["Onshore"]  +
            df_intermittent["Offshore"]  +
            df_intermittent["PV"]  +
            df_intermittent["Run_of_river"] 
        )

        area_demand = df_demand[area]
        df_residual[area] = area_demand - df_reserve_intermittent[area]
        

    df_residual = df_residual.round(4) 
    residual_output_file = os.path.join(output_folder, "Residual_load.csv")
    df_residual.to_csv(residual_output_file)
    print("Done!")


residual_load(
    intermittent_folder=Path("2024_Intermittent"),
    demand_file=Path("2024_Data/Consumption_2024.csv"),
    output_folder=Path("Reserve_req_2024"),
)

# Calc beta - common to al scenarios

In [ ]:

def calc_beta(totalhydro, k_file, residual_file, output_folder):
    areas = ["NO1", "NO2", "NO3", "NO4", "NO5", "SE1", "SE2", "SE3", "SE4"]
    
    df_totalhydro = pd.read_csv(totalhydro).set_index("Area")
    df_residual = pd.read_csv(residual_file, index_col="Timestamp")
    df_k = pd.read_csv(k_file, index_col="Area")
    df_beta = pd.DataFrame(index=df_residual.index)

    for area in areas:
        water_energy = df_totalhydro.loc[area, 'Installert effekt [MW]']
        k_energy = df_k.loc[area, "Thermal"] + df_k.loc[area, "Nuclear"]
        
        available_energy = water_energy + k_energy
        
        df_beta[area] = df_residual[area] / available_energy

    df_beta = df_beta.round(4)

    beta_output_file = os.path.join(output_folder, "Beta.csv")
    df_beta.to_csv(beta_output_file)

    print(f"File saved to {beta_output_file}")


calc_beta(
    totalhydro=Path("2024_Data/Hydro_info_2024.csv"),
    k_file = Path("2024_Data/K_energy_2024.csv"),
    residual_file = Path("Reserve_req_2024/Residual_load.csv"),
    output_folder=Path("Reserve_req_2024")
)

# Make sure beta is 0 <= b <= 1

In [ ]:
def calc_reserve_beta(beta_file, output_folder):
    df_beta = pd.read_csv(beta_file, index_col="Timestamp")

    df_beta[df_beta <= 0] = 0
    df_beta[df_beta >= 1] = 1

    beta_output_file = os.path.join(output_folder, "Beta_reserve.csv")
    df_beta.to_csv(beta_output_file)

calc_reserve_beta(
    beta_file=Path("Reserve_req_2024/Beta.csv"),
    output_folder=Path("Reserve_req_2024"),
)


# Calc final reserve req (req * beta) for BaseCase and UpperBound

In [ ]:
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt

def final_reserve_req_multiple(scenarios, input_dir, output_dir):
    areas = ["NO1", "NO2", "NO3", "NO4", "NO5", "SE1", "SE2", "SE3", "SE4"]

    for scenario in scenarios:
        print(f"Processing scenario: {scenario}")
        
        up_path = input_dir / f"Up_reserve_{scenario}.csv"
        down_path = input_dir / f"Down_reserve_{scenario}.csv"
        beta_path = input_dir / f"Beta_reserve.csv"

        df_up_req = pd.read_csv(up_path, index_col="Timestamp")
        df_down_req = pd.read_csv(down_path, index_col="Timestamp")
        df_beta = pd.read_csv(beta_path, index_col="Timestamp")

        up = pd.DataFrame(index=df_up_req.index)
        down = pd.DataFrame(index=df_down_req.index)
        for area in areas:
            up[area] = df_up_req[area] * df_beta[area]
            down[area] = df_down_req[area] * df_beta[area]

        up = up.round(4)
        down = down.round(4)

    
        up_output_file = output_dir / f"Final_up_reserve_{scenario}.csv"
        down_output_file = output_dir / f"Final_down_reserve_{scenario}.csv"
        up.to_csv(up_output_file)
        down.to_csv(down_output_file)



final_reserve_req_multiple(
    scenarios=["UpperBound", "BaseCase"],
    input_dir=Path("Reserve_req_2024"),
    output_dir=Path("2024_Data")
)


# Calc reserve requirements for LowerBound - one common req for all areas

In [ ]:
import pandas as pd
import os
from pathlib import Path

def reserve_total_per_area_beta(intermittent_folder, demand_file, beta_file, output_folder, v, x, y, z, d):
    areas = ["NO1", "NO2", "NO3", "NO4", "NO5", "SE1", "SE2", "SE3", "SE4"]

    df_demand = pd.read_csv(demand_file, index_col="Timestamp")
    df_demand.index = df_demand.index.astype(int)

    df_beta = pd.read_csv(beta_file, index_col="Timestamp")
    df_beta.index = df_beta.index.astype(int)

    total_reserve = pd.Series(0.0, index=df_demand.index)

    for area in areas:
        intermittent_file = os.path.join(intermittent_folder, f"{area}_intermittent_energy.csv")
        df_intermittent = pd.read_csv(intermittent_file, index_col="Timestamp")
        df_intermittent.index = df_intermittent.index.astype(int)

        intermittent_reserve = (
            df_intermittent["Onshore"] * v +
            df_intermittent["Offshore"] * x +
            df_intermittent["PV"] * y +
            df_intermittent["Run_of_river"] * z
        )

        demand_adjusted = df_demand[area] * d
        beta = df_beta[area]

        combined = (intermittent_reserve + demand_adjusted) * beta
        total_reserve += combined

    up_reserve = total_reserve + 1450       #Highest RI for up-regulation
    down_reserve = total_reserve + 1400     # Highest RI for down-regulation 

    up_reserve[up_reserve < 0] = 0
    down_reserve[down_reserve < 0] = 0

    df_up = pd.DataFrame({"Total_Up_Reserve": up_reserve.round(2)})
    df_down = pd.DataFrame({"Total_Down_Reserve": down_reserve.round(2)})

    df_up.to_csv(os.path.join(output_folder, "Final_up_reserve_LowerBound.csv"))
    df_down.to_csv(os.path.join(output_folder, "Final_down_reserve_LowerBound.csv"))


reserve_total_per_area_beta(
    intermittent_folder=Path("2024_Intermittent"),
    demand_file=Path("2024_Data/Consumption_2024.csv"),
    beta_file=Path("Reserve_req_2024/Beta_reserve.csv"),
    output_folder=Path("2024_Data"),
    v=0.1449,           # Share of Onshore 
    x=0.1993,           # Share of offshore
    y=0.1696,           # Share of PV
    z=0.01,             # Share of Run_of_river
    d=0.00825,          # Share of demand 
)